In [1]:
import sympy
from sympy import symbols, Matrix, Function, sin, cos, zeros, eye, simplify, lambdify, diag, Eq, solve, pi
import numpy as np

In [5]:
## Helper Functions 

## vector to se3
def unhat6(hatted):
    vec = Matrix([hatted[0,3], hatted[1,3], hatted[2,3], hatted[2,1], hatted[0,2], hatted[1,0]])
    return vec

## se3 from vector
def hat6(unhatted):
    hat = zeros(4,4)
    v1 = unhatted[0, 0]
    v2 = unhatted[1, 0]
    v3 = unhatted[2, 0]
    w1 = unhatted[3,0]
    w2 = unhatted[4,0]
    w3 = hatted[5,0]    
    mat = Matrix([[0, -w3, w2, v1],[w3, 0, -w1, v2],[-w2, w1, 0, v3],[0, 0, 0, 0]])
    return mat

## Un-homogenizing a vector
def unbar4(barred):
    vec = barred[:3]
    return vec

## inverse of SE3 transform
def SE3Inv(g):
    inv = zeros(4,4)
    inv[:3, :3] = g[:3,:3].T
    inv[:3, 3] = -g[:3,:3].T * g[:3, 3]
    inv[3, :] = g[3,:]
    return inv

## time derivative of SE3 transform
def SE3dt(SE3, t):
    return SE3.diff(t)

def toG(ang, x, y):
    return Matrix([[cos(ang), -sin(ang), 0, x],[sin(ang), cos(ang), 0, y],[0, 0, 1, 0],[0,0,0,1]])


In [31]:
## system parameters
M, m, L, R, g, W, t = symbols("M, m, L, R, g, W, t")

## systems variables
theta = Function("theta")(t)
phi = Function("phi")(t)

q = Matrix([theta, phi])
qdot = q.diff(t)
qddot = qdot.diff(t)

thetadot = qdot[0]
thetaddot = qddot[0]
phidot = qdot[1]
phiddot = qddot[1]

## Setting up transformation of COM Frames wrt inertial frame {W}
gWA = toG(0, phi*R, R)
gAB = toG(theta, 0, 0)
gBC = toG(0, 0, L)  ## THERE MIGHT BE AN OFSET FOR THE C.O.M IN THE X DIRECTION 

gWC = gWA*gAB*gBC

## Velocities
V_m = unhat6(SE3Inv(gWA) * SE3dt(gWA, t))
V_M = unhat6(SE3Inv(gWC) * SE3dt(gWC, t))
# display("V_m, V_M", V_m, V_M)

## MassInertia Matrices
I_m, I_M = symbols("I_m, I_M")
MI_m = diag(m, m, m, 0, 0, I_m)
MI_M = diag(M, M, M, 0, 0, I_M)
display(MI_m)

## Kinetic Energies
KE_m = (1/2 * V_m.T * MI_m * V_m)[0]
KE_M = (1/2 * V_M.T * MI_M * V_M)[0]
display("KE_m, KE_M", KE_m, KE_M)

## Potential Energies
PE_m = (m*g*gWA * Matrix([0,0,0,1]))[1]
PE_M = (M*g*gWC * Matrix([0,0,0,1]))[1]
display("PE_m, PE_M", PE_m, PE_M)



Matrix([
[m, 0, 0, 0, 0,   0],
[0, m, 0, 0, 0,   0],
[0, 0, m, 0, 0,   0],
[0, 0, 0, 0, 0,   0],
[0, 0, 0, 0, 0,   0],
[0, 0, 0, 0, 0, I_m]])

'KE_m, KE_M'

0.5*R**2*m*Derivative(phi(t), t)**2

I_M*(0.5*sin(theta(t))**2*Derivative(theta(t), t) + 0.5*cos(theta(t))**2*Derivative(theta(t), t))*(sin(theta(t))**2*Derivative(theta(t), t) + cos(theta(t))**2*Derivative(theta(t), t)) + M*(-L*sin(theta(t))**2*Derivative(theta(t), t) + (-L*cos(theta(t))*Derivative(theta(t), t) + R*Derivative(phi(t), t))*cos(theta(t)))*(-0.5*L*sin(theta(t))**2*Derivative(theta(t), t) + 0.5*(-L*cos(theta(t))*Derivative(theta(t), t) + R*Derivative(phi(t), t))*cos(theta(t))) + M*(-L*sin(theta(t))*cos(theta(t))*Derivative(theta(t), t) - (-L*cos(theta(t))*Derivative(theta(t), t) + R*Derivative(phi(t), t))*sin(theta(t)))*(-0.5*L*sin(theta(t))*cos(theta(t))*Derivative(theta(t), t) - 0.5*(-L*cos(theta(t))*Derivative(theta(t), t) + R*Derivative(phi(t), t))*sin(theta(t)))

'PE_m, PE_M'

R*g*m

M*g*(L*cos(theta(t)) + R)

In [29]:
## Lagrangian
L = simplify(KE_m + KE_M - (PE_m + PE_M))
display("L", L)

## Formulating the unconstrained E-L expression
dLdq = L.diff(q)
dLdqdot = L.diff(qdot)
d_dt = dLdqdot.diff(t)
EL = simplify(d_dt - dLdq)
# print("Unconstrained Euler Lagrange expression:")
# display(EL)

# H = (dLdqdot.T * qdot)[0] - L
# print("Hamiltonian:")
# display(simplify(H))

'L'

0.5*I_M*Derivative(theta(t), t)**2 + 0.5*L**2*M*Derivative(theta(t), t)**2 - 1.0*L*M*R*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) - 1.0*L*M*g*cos(theta(t)) + 0.5*M*R**2*Derivative(phi(t), t)**2 - 1.0*M*R*g + 0.5*R**2*m*Derivative(phi(t), t)**2 - 1.0*R*g*m

In [28]:
## Formulating the EL eqns with generalized external forces
## Torque on wheels +tau; equal and opposite force on body
## Discounting frictions between wheel-motor and wheel-ground

tau = Function("tau")(t)
_tau = symbols("tau")
Forces =  Matrix([_tau, -_tau])
EL_eqns = Eq(EL, Forces)
# display(EL_eqns)


## Solving for Accelerations
solns = solve(EL_eqns, qddot)
# display(solns)

theta_dot = solns[qddot[0]]
phi_dot = solns[qddot[1]]
display("thetadot", theta_dot)
display("phidot", phi_dot)

'thetadot'

(L*M*(L*M*R*sin(theta(t))*Derivative(theta(t), t)**2 + tau)*cos(theta(t)) - R*(M + m)*(L*M*g*sin(theta(t)) + tau))/(R*(L**2*M**2*cos(theta(t))**2 - (I_M + L**2*M)*(M + m)))

'phidot'

(-L*M*R*(L*M*g*sin(theta(t)) + tau)*cos(theta(t)) + (I_M + L**2*M)*(L*M*R*sin(theta(t))*Derivative(theta(t), t)**2 + tau))/(R**2*(L**2*M**2*cos(theta(t))**2 - (I_M + L**2*M)*(M + m)))

In [ ]:
## Solving for the theta/phi transfer function
## need to linearize; equate taus, rearrangeve